In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [2]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=64

# create sample

In [4]:
%cd data

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data


In [5]:
%cd train

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/train


In [6]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

mkdir: cannot create directory ‘../sample’: File exists


In [7]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [8]:
from shutil import copyfile

In [9]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [10]:
%cd ../valid

/home/ubuntu/courses/deeplearning1/kaggle/statefarm/data/valid


In [11]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [12]:
%cd ../../..

/home/ubuntu/courses/deeplearning1/kaggle


In [13]:
%cd statefarm

/home/ubuntu/courses/deeplearning1/kaggle/statefarm


In [14]:
%mkdir data/results

mkdir: cannot create directory ‘data/results’: File exists


In [15]:
%mkdir data/sample/test

# Create batches

In [16]:
path = "data/sample/"

In [17]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [18]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

In [19]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

In [20]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 13.5930 - acc: 0.1187 - val_loss: 14.0257 - val_acc: 0.1210
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 13.7547 - acc: 0.1427 - val_loss: 14.0438 - val_acc: 0.1280


In [21]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_1[0][0]                  
Total params: 1505296
____________________________________________________________________________________________________


In [23]:
10*3*224*224

1505280

The loss is pretty high.  Let's try predicting 10 sample and round it to the nearest 2 decimal places.

As we can see below, it's predicting 1 or 6.   
we'll lower the learning rate.

In [22]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [23]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 2.3660 - acc: 0.1960 - val_loss: 4.6861 - val_acc: 0.1660
Epoch 2/2
1500/1500 [==============================] - 25s - loss: 1.7346 - acc: 0.4340 - val_loss: 3.0217 - val_acc: 0.2530


In [24]:
model.optimizer.lr=0.001

In [25]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 1.3645 - acc: 0.5953 - val_loss: 2.5782 - val_acc: 0.2680
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 1.1118 - acc: 0.7180 - val_loss: 2.1493 - val_acc: 0.3060


In [26]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 32s - loss: 0.9362 - acc: 0.7867 - val_loss: 1.9666 - val_acc: 0.3410
Epoch 2/2
1500/1500 [==============================] - 25s - loss: 0.7970 - acc: 0.8480 - val_loss: 1.9930 - val_acc: 0.3370


In [27]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [28]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 1.99,  0.34],
       [ 2.01,  0.33],
       [ 2.01,  0.33],
       [ 1.97,  0.33],
       [ 2.  ,  0.35],
       [ 1.96,  0.35],
       [ 2.  ,  0.34],
       [ 2.01,  0.33],
       [ 1.99,  0.34],
       [ 1.97,  0.34]])

### Single conv layer

In [29]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [30]:
conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 35s - loss: 1.4018 - acc: 0.5933 - val_loss: 2.0850 - val_acc: 0.2750
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 0.3166 - acc: 0.9420 - val_loss: 1.9836 - val_acc: 0.3290
Epoch 1/4
1500/1500 [==============================] - 41s - loss: 0.0879 - acc: 0.9933 - val_loss: 2.0928 - val_acc: 0.2310
Epoch 2/4
1500/1500 [==============================] - 31s - loss: 0.0447 - acc: 0.9980 - val_loss: 2.1729 - val_acc: 0.2140
Epoch 3/4
1500/1500 [==============================] - 33s - loss: 0.0224 - acc: 1.0000 - val_loss: 2.2115 - val_acc: 0.2060
Epoch 4/4
1500/1500 [==============================] - 28s - loss: 0.0145 - acc: 1.0000 - val_loss: 2.2399 - val_acc: 0.1850


## Data augmentation
try data augmentation to avoid overfitting.    Our accurancy is way too high.   and val_acc is way too low.

In [33]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [34]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 37s - loss: 2.5248 - acc: 0.2107 - val_loss: 3.1806 - val_acc: 0.1130
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 1.8088 - acc: 0.3780 - val_loss: 2.1439 - val_acc: 0.2740
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 1.5604 - acc: 0.4853 - val_loss: 2.0216 - val_acc: 0.3310
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 1.3389 - acc: 0.5527 - val_loss: 2.0223 - val_acc: 0.3220
Epoch 3/4
1500/1500 [==============================] - 30s - loss: 1.3135 - acc: 0.5660 - val_loss: 2.1241 - val_acc: 0.2890
Epoch 4/4
1500/1500 [==============================] - 32s - loss: 1.1495 - acc: 0.6293 - val_loss: 2.2366 - val_acc: 0.2260


In [35]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 35s - loss: 1.1400 - acc: 0.6380 - val_loss: 2.1175 - val_acc: 0.2920
Epoch 2/5
1500/1500 [==============================] - 31s - loss: 1.0407 - acc: 0.6747 - val_loss: 2.1249 - val_acc: 0.2570
Epoch 3/5
1500/1500 [==============================] - 28s - loss: 0.9242 - acc: 0.7013 - val_loss: 1.8565 - val_acc: 0.3510
Epoch 4/5
1500/1500 [==============================] - 32s - loss: 0.8863 - acc: 0.7187 - val_loss: 1.6105 - val_acc: 0.4620
Epoch 5/5
1500/1500 [==============================] - 27s - loss: 0.8956 - acc: 0.7073 - val_loss: 1.5919 - val_acc: 0.4640
